# EDA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
src_path = os.path.join(module_path, "src")
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(src_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [2]:
from src.recommenders import MainRecommender

In [3]:
data = pd.read_csv('../data/retail_train.csv')
item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

## retail_train

Таблица взаимодействий. Клиенты, заказы, товары, количество, цены, время, скидки

In [4]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [5]:
data.tail()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2396799,1613,41655820646,663,16102849,1,2.00,3262,-1.15,1231,95,0.0,0.0
2396800,1001,41655829421,663,13217063,1,1.69,3131,0.00,2231,95,0.0,0.0
2396801,1001,41655829421,663,13217800,1,1.69,3131,0.00,2231,95,0.0,0.0
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0
2396803,1167,41656790510,663,6430664,1,8.00,3385,0.00,1059,95,0.0,0.0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2396804 entries, 0 to 2396803
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   user_id            int64  
 1   basket_id          int64  
 2   day                int64  
 3   item_id            int64  
 4   quantity           int64  
 5   sales_value        float64
 6   store_id           int64  
 7   retail_disc        float64
 8   trans_time         int64  
 9   week_no            int64  
 10  coupon_disc        float64
 11  coupon_match_disc  float64
dtypes: float64(4), int64(8)
memory usage: 219.4 MB


Числовые значения, пропусков нет, категориальных нет (есть числовые идентификаторы клиентов, заказов, товаров). 

In [7]:
data.describe()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
count,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06,2.396804e+06
mean,1.271904e+03,3.336643e+10,3.639770e+02,2.827247e+06,1.003763e+02,3.100821e+00,3.048227e+03,-5.400708e-01,1.561714e+03,5.268156e+01,-1.638696e-02,-2.897905e-03
std,7.265644e+02,4.284798e+09,1.759385e+02,3.732798e+06,1.152379e+03,4.210229e+00,8.785542e+03,1.245824e+00,4.015691e+02,2.513310e+01,2.168615e-01,3.974618e-02
min,1.000000e+00,2.698485e+10,1.000000e+00,2.567100e+04,0.000000e+00,0.000000e+00,1.000000e+00,-1.300200e+02,0.000000e+00,1.000000e+00,-5.593000e+01,-7.700000e+00
25%,6.550000e+02,3.008714e+10,2.160000e+02,9.169930e+05,1.000000e+00,1.290000e+00,3.300000e+02,-6.900000e-01,1.307000e+03,3.200000e+01,0.000000e+00,0.000000e+00
50%,1.271000e+03,3.241998e+10,3.660000e+02,1.027569e+06,1.000000e+00,2.000000e+00,3.700000e+02,-2.000000e-02,1.614000e+03,5.300000e+01,0.000000e+00,0.000000e+00
75%,1.914000e+03,3.514580e+10,5.150000e+02,1.132178e+06,1.000000e+00,3.490000e+00,4.220000e+02,0.000000e+00,1.844000e+03,7.400000e+01,0.000000e+00,0.000000e+00
max,2.500000e+03,4.165679e+10,6.630000e+02,1.802456e+07,8.963800e+04,8.400000e+02,3.428000e+04,3.990000e+00,2.359000e+03,9.500000e+01,0.000000e+00,0.000000e+00


## product

Описание товаров. Идентификаторы товаров, производителей, подразделение на отделы, локальность, описание

In [8]:
item_features.head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [9]:
item_features.tail()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
92348,18293142,6384,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,
92349,18293439,6393,DRUG GM,National,BOOKSTORE,CHILDRENS LOW END,
92350,18293696,6406,DRUG GM,National,BOOKSTORE,PAPERBACK BEST SELLER,
92351,18294080,6442,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,
92352,18316298,764,GROCERY,National,PAPER TOWELS,PAPER TOWELS & HOLDERS,


In [10]:
item_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92353 entries, 0 to 92352
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   item_id               92353 non-null  int64 
 1   manufacturer          92353 non-null  int64 
 2   department            92353 non-null  object
 3   brand                 92353 non-null  object
 4   commodity_desc        92353 non-null  object
 5   sub_commodity_desc    92353 non-null  object
 6   curr_size_of_product  92353 non-null  object
dtypes: int64(2), object(5)
memory usage: 4.9+ MB


Нет пропусков, но в строковых описаниях в столбце "urr_size_of_product" есть пустые строки

In [11]:
item_features.describe()

,item_id,manufacturer
count,9.235300e+04,92353.00000
mean,5.328353e+06,1739.22833
std,5.359937e+06,1818.26957
min,2.567100e+04,1.00000
25%,9.706280e+05,328.00000
50%,1.621091e+06,1094.00000
75%,9.704770e+06,2264.00000
max,1.831630e+07,6477.00000


In [12]:
def print_categorical(data):
    for cat_colname in data.columns:
        if data[cat_colname].nunique() < 310:
            print(str(cat_colname) + '\n\n' + str(data[cat_colname].unique()) + '\n\n')
            print(data.groupby(cat_colname)[cat_colname].count())
            print('*' * 100 + '\n')

In [13]:
print_categorical(item_features)

department

['GROCERY' 'MISC. TRANS.' 'PASTRY' 'DRUG GM' 'MEAT-PCKGD' 'SEAFOOD-PCKGD'
 'PRODUCE' 'NUTRITION' 'DELI' 'COSMETICS' 'MEAT' 'FLORAL'
 'TRAVEL & LEISUR' 'SEAFOOD' 'MISC SALES TRAN' 'SALAD BAR' 'KIOSK-GAS'
 'ELECT &PLUMBING' 'GRO BAKERY' 'GM MERCH EXP' 'FROZEN GROCERY'
 'COUP/STR & MFG' 'SPIRITS' 'GARDEN CENTER' 'TOYS' 'CHARITABLE CONT'
 'RESTAURANT' 'RX' 'PROD-WHS SALES' 'MEAT-WHSE' 'DAIRY DELI' 'CHEF SHOPPE'
 'HBC' 'DELI/SNACK BAR' 'PORK' 'AUTOMOTIVE' 'VIDEO RENTAL' ' '
 'CNTRL/STORE SUP' 'HOUSEWARES' 'POSTAL CENTER' 'PHOTO' 'VIDEO'
 'PHARMACY SUPPLY']


department
                      15
AUTOMOTIVE             2
CHARITABLE CONT        2
CHEF SHOPPE           14
CNTRL/STORE SUP        4
COSMETICS           3011
COUP/STR & MFG        39
DAIRY DELI             3
DELI                2354
DELI/SNACK BAR         2
DRUG GM            31529
ELECT &PLUMBING        1
FLORAL               938
FROZEN GROCERY        23
GARDEN CENTER        128
GM MERCH EXP           3
GRO BAKERY       

In [14]:
item_features['sub_commodity_desc'].unique()

array(['ICE - CRUSHED/CUBED', 'NO SUBCOMMODITY DESCRIPTION',
       'BREAD:ITALIAN/FRENCH', ..., 'SEMI MOIST CAT FOOD: BOX',
       'BERRIES OTHER', 'ROSES OTHER'], dtype=object)

In [15]:
item_features['sub_commodity_desc'].nunique()

2383

In [16]:
item_features['curr_size_of_product'].unique()

array(['22 LB', ' ', '50 OZ', ..., '6.3 IN', '35 LD', '2 LTR PET'],
      dtype=object)

In [17]:
item_features['curr_size_of_product'].nunique()

4345

## hh_demographic

Описание клиентов. Идентификаторы клиентов, возрастные группы, семейное положение, группы по доходу, данные по недвижимости и количеству детей

In [18]:
user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [19]:
user_features.tail()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
796,35-44,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2494
797,45-54,A,75-99K,Homeowner,Unknown,3,1,2496
798,45-54,U,35-49K,Unknown,Single Male,1,None/Unknown,2497
799,25-34,U,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,2498
800,25-34,U,Under 15K,Unknown,2 Adults Kids,3,1,2499


In [20]:
user_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   age_desc             801 non-null    object
 1   marital_status_code  801 non-null    object
 2   income_desc          801 non-null    object
 3   homeowner_desc       801 non-null    object
 4   hh_comp_desc         801 non-null    object
 5   household_size_desc  801 non-null    object
 6   kid_category_desc    801 non-null    object
 7   user_id              801 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 50.2+ KB


In [21]:
data.user_id.nunique()

2499

In [22]:
f"{user_features.user_id.nunique()/data.user_id.nunique()*100:.2f}%"

'32.05%'

In [23]:
u_all = set(data.user_id.unique().tolist())
u_inf = set(user_features.user_id.unique().tolist())

len(u_all.intersection(u_inf))

801

Пропусков нет, но данные охватывают только 32.05% всех пользователей

# Фильтрация

### Фильтруем 3 самых популярных

In [24]:
popularity = data.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [25]:
item_features.loc[item_features['item_id'].isin(top_5000[:5])]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
44796,1404121,69,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
57181,6533889,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
57216,6534166,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
57221,6534178,69,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
57335,6544236,69,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,


In [26]:
data.loc[data['item_id'].isin(top_5000[:3]), 'item_id'] = 999999

### Уберем товары, которые не продавались за последние 12 месяцев

In [27]:
curr_week = data['week_no'].values.max()
data.loc[data['week_no']<curr_week-53, 'item_id'] = 999999

### Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоят < 1$

In [28]:
mean_price = data.groupby('item_id')['quantity', "sales_value"].sum()
mean_price["mean_price"] = mean_price["sales_value"] / mean_price["quantity"]
cheap_list = mean_price.loc[mean_price["mean_price"]<0.9].index.tolist()
data.loc[data['item_id'].isin(cheap_list), 'item_id'] = 999999

### Cамые непопулярные

In [29]:
popularity_pu = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity_pu.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
popularity_pu.sort_values('share_unique_users', ascending=False, inplace=True)
popularity_pu.tail()

,item_id,share_unique_users
34338,2039932,1
34337,2039917,1
34336,2039709,1
34335,2039684,1
32131,1381417,1


In [30]:
popularity_pu = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity_pu.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
popularity_pu['share_unique_users'] = popularity_pu['share_unique_users'] / data['user_id'].nunique()
popularity_pu.sort_values('share_unique_users', ascending=False, inplace=True)
popularity_pu.head()

,item_id,share_unique_users
18639,999999,1.000000
25733,1082185,0.724690
17067,981760,0.529012
18195,995242,0.469388
21204,1029743,0.452581


In [31]:
popularity_pu.shape

(64262, 2)

In [32]:
popularity_pu.tail()

,item_id,share_unique_users
34338,2039932,0.0004
34337,2039917,0.0004
34336,2039709,0.0004
34335,2039684,0.0004
32131,1381417,0.0004


In [33]:
popularity_pu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64262 entries, 18639 to 32131
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   item_id             64262 non-null  int64  
 1   share_unique_users  64262 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.5 MB


In [34]:
popularity_pu[popularity_pu['share_unique_users'] < 0.005]

,item_id,share_unique_users
25257,1076889,0.004802
50649,12257397,0.004802
4101,831809,0.004802
59682,14111571,0.004802
28637,1116451,0.004802
...,...,...
34338,2039932,0.000400
34337,2039917,0.000400
34336,2039709,0.000400
34335,2039684,0.000400


In [35]:
unpop_list = popularity_pu[popularity_pu['share_unique_users'] < 0.005].item_id.tolist()
data.loc[data['item_id'].isin(unpop_list), 'item_id'] = 999999

### Не интересные для рекоммендаций категории (department)

In [36]:
item_features.shape

(92353, 7)

In [37]:
item_features['department'].unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [38]:
item_features[item_features['department'] == 'PROD-WHS SALES']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
10781,862237,2,PROD-WHS SALES,National,PROD SUPPLIES,TICKETS,
30852,1040186,2,PROD-WHS SALES,National,PROD SUPPLIES,COUPON,


In [39]:
item_features[item_features['department'] == 'RX']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
10776,862185,5983,RX,National,PHARMACY,MENSTRUAL ANALGESICS,30 CT
23890,977999,6073,RX,National,PHARMACY,JHOOK - PERSONAL CARE,
30923,1040789,5720,RX,National,PHARMACY,FIRST AID MEDICATIONS,
33043,1059903,125,RX,National,PHARMACY,N-PRESC HEARING AID BATTERIES,4 CT
42459,1185885,5,RX,Private,COUPON/MISC ITEMS,MISC SALES TRANS,
44511,1377652,2151,RX,National,PHARMACY,CONVALESCENT CARE - CANES,1 CT
52595,5565609,569,RX,National,PHARMACY,RX OTC BEAUTYASSESSPILL BX/H,
66044,9368445,5059,RX,National,PHARMACY,COLD AND FLU - DXM,40 CT
79409,12810538,851,RX,National,PHARMACY,N-PRESC HEARING AID BATTERIES,16 CT


In [40]:
item_features[item_features['department'] == 'MEAT-WHSE']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
12746,879637,5995,MEAT-WHSE,National,COUPON,MISC SALES TRANS,


In [41]:
item_features[item_features['department'] == 'HBC']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
21478,956174,177,HBC,National,INFANT CARE PRODUCTS,FEEDING ACCESSORIES BOTTLES,


In [42]:
item_features[item_features['department'] == 'CNTRL/STORE SUP']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
59612,7100347,1871,CNTRL/STORE SUP,National,COUPON/MISC ITEMS,CENTRAL SUPPLIES,125 CT
78400,12671292,5879,CNTRL/STORE SUP,National,COUPON/MISC ITEMS,COFF SHOP: RETAIL PACK BEVERAG,1 LB
79932,13004709,6011,CNTRL/STORE SUP,National,COUPON/MISC ITEMS,CENTRAL SUPPLIES,
91676,17330423,903,CNTRL/STORE SUP,Private,COUPON/MISC ITEMS,ELECTRONIC GIFT CARDS ACTIVATI,


In [43]:
item_features[item_features['department'] == 'POSTAL CENTER']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
64268,8293561,693,POSTAL CENTER,National,CANDY - PACKAGED,CANDY BAGS-CHOCOCLATE,12 PK
91025,17179638,693,POSTAL CENTER,National,CANDY - PACKAGED,SEASONAL CANDY BOX-CHOCOLATE,
92256,18143349,857,POSTAL CENTER,National,CANDY - CHECKLANE,CANDY BARS (SINGLES)(INCLUDING,


In [44]:
item_features[item_features['department'] == 'PHOTO']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
79669,12949577,1225,PHOTO,National,REFRGRATD DOUGH PRODUCTS,REFRIGERATED COOKIES-SEASONAL,18 OZ
85781,14077294,236,PHOTO,National,REFRGRATD DOUGH PRODUCTS,MISC REFRIG DOUGH PRODUCTS,8 OZ


In [45]:
item_features[item_features['department'] == 'VIDEO']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
85754,14073108,6375,VIDEO,National,BEERS/ALES,BEERALEMALT LIQUORS,
87652,15681643,239,VIDEO,National,BEERS/ALES,BEERALEMALT LIQUORS,


In [46]:
item_features[item_features['department'] == 'PHARMACY SUPPLY']

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
86113,15449615,6129,PHARMACY SUPPLY,National,COUPON/MISC ITEMS,JUICE,13.5 OZ


In [47]:
dep_list = ['PROD-WHS SALES', 'RX', 'MEAT-WHSE', 
            'HBC', 'CNTRL/STORE SUP', 'POSTAL CENTER', 
            'PHOTO', 'VIDEO', 'PHARMACY SUPPLY']

In [48]:
for dep in dep_list:
    print(item_features.loc[item_features['department'] == dep, "item_id"].values)

[ 862237 1040186]
[  862185   977999  1040789  1059903  1185885  1377652  5565609  9368445
 12810538]
[879637]
[956174]
[ 7100347 12671292 13004709 17330423]
[ 8293561 17179638 18143349]
[12949577 14077294]
[14073108 15681643]
[15449615]


In [49]:
for dep in dep_list:
    drop_items_list = item_features.loc[item_features['department'] == dep, "item_id"].values
    sl = data.loc[data['item_id'].isin(drop_items_list), 'item_id'].count()
    print(f"dep: {dep} sale count: {sl}")
    data.loc[data['item_id'].isin(drop_items_list), 'item_id'] = 999999

dep: PROD-WHS SALES sale count: 0
dep: RX sale count: 0
dep: MEAT-WHSE sale count: 0
dep: HBC sale count: 0
dep: CNTRL/STORE SUP sale count: 0
dep: POSTAL CENTER sale count: 0
dep: PHOTO sale count: 0
dep: VIDEO sale count: 0
dep: PHARMACY SUPPLY sale count: 0


## Feature Engineering

### Средний чек

In [50]:
bills = data.groupby(["basket_id", 'user_id'])["sales_value"].sum()
bills

basket_id    user_id
26984851472  2375        5.91
26984851516  2375       12.43
26984896261  1364       11.37
26984905972  1130        2.55
26984945254  1173        3.53
                        ...  
41655046833  2438       29.11
41655180249  2364       29.53
41655820646  1613       45.04
41655829421  1001        3.38
41656790510  1167       51.98
Name: sales_value, Length: 257001, dtype: float64

In [51]:
bills = bills.reset_index(level=[0,1])
bills.head()

,basket_id,user_id,sales_value
0,26984851472,2375,5.91
1,26984851516,2375,12.43
2,26984896261,1364,11.37
3,26984905972,1130,2.55
4,26984945254,1173,3.53


In [52]:
bills_avg = data.groupby(['user_id'])["basket_id"].count()
bills_avg

user_id
1       1589
2        655
3        889
4        301
5        219
        ... 
2496    1420
2497    1938
2498     833
2499     969
2500    1331
Name: basket_id, Length: 2499, dtype: int64

In [53]:
bills_avg = bills_avg.reset_index()
bills_avg.columns=['user_id', 'bills_cnt']
bills_avg

,user_id,bills_cnt
0,1,1589
1,2,655
2,3,889
3,4,301
4,5,219
...,...,...
2494,2496,1420
2495,2497,1938
2496,2498,833
2497,2499,969


In [54]:
bills_sum = bills.groupby(['user_id'])["sales_value"].sum()
bills_sum

user_id
1       3959.91
2       1823.45
3       2594.30
4       1200.11
5        749.09
         ...   
2496    4105.29
2497    6848.67
2498    2509.39
2499    2874.39
2500    4937.16
Name: sales_value, Length: 2499, dtype: float64

In [55]:
bills_avg['bill_sum'] = bills_sum.values
bills_avg.head()

,user_id,bills_cnt,bill_sum
0,1,1589,3959.91
1,2,655,1823.45
2,3,889,2594.30
3,4,301,1200.11
4,5,219,749.09


In [56]:
bills_avg['bill_avg'] = bills_avg['bill_sum'] / bills_avg['bills_cnt']
bills_avg.head()

,user_id,bills_cnt,bill_sum,bill_avg
0,1,1589,3959.91,2.492077
1,2,655,1823.45,2.783893
2,3,889,2594.30,2.918223
3,4,301,1200.11,3.987076
4,5,219,749.09,3.420502


In [57]:
user_features = user_features.merge(bills_avg[["user_id", "bill_avg"]], on='user_id', how='left')
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,bill_avg
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,2.492077
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,2.673405


### Когда была крайняя покупка

In [58]:
bills_per_week = data.groupby(['user_id'])["week_no"].max()
bills_per_week

user_id
1       95
2       90
3       92
4       90
5       85
        ..
2496    95
2497    95
2498    95
2499    92
2500    95
Name: week_no, Length: 2499, dtype: int64

In [59]:
bills_per_week = bills_per_week.reset_index()
bills_per_week.columns = ["user_id", "week_last_bill"]
bills_per_week

,user_id,week_last_bill
0,1,95
1,2,90
2,3,92
3,4,90
4,5,85
...,...,...
2494,2496,95
2495,2497,95
2496,2498,95
2497,2499,92


In [60]:
user_features = user_features.merge(bills_per_week, on='user_id', how='left')
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,bill_avg,week_last_bill
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,2.492077,95
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,2.673405,95


### Количество покупок в неделю

In [61]:
bills_week = data.groupby(['user_id', 'week_no'])["item_id"].count()
bills_week

user_id  week_no
1        8          30
         10         12
         13         12
         14         23
         15         17
                    ..
2500     91         11
         92         25
         93         26
         94         19
         95         36
Name: item_id, Length: 114839, dtype: int64

In [62]:
bills_week = bills_week.reset_index()
bills_week.columns = ["user_id", "week_no", "item_cnt"]
bills_week.head()

,user_id,week_no,item_cnt
0,1,8,30
1,1,10,12
2,1,13,12
3,1,14,23
4,1,15,17


In [63]:
bills_count = bills_week.groupby("user_id")["item_cnt"].sum()
bills_count

user_id
1       1589
2        655
3        889
4        301
5        219
        ... 
2496    1420
2497    1938
2498     833
2499     969
2500    1331
Name: item_cnt, Length: 2499, dtype: int64

In [64]:
bills_per_week['item_cnt'] = bills_count.values

In [65]:
old_bill = data.groupby(['user_id'])["week_no"].min()
old_bill

user_id
1        8
2       15
3       17
4       16
5       13
        ..
2496    17
2497    12
2498    16
2499    11
2500    12
Name: week_no, Length: 2499, dtype: int64

In [66]:
bills_per_week["week_oldest_bill"] = old_bill.values
bills_per_week.head()

,user_id,week_last_bill,item_cnt,week_oldest_bill
0,1,95,1589,8
1,2,90,655,15
2,3,92,889,17
3,4,90,301,16
4,5,85,219,13


In [67]:
bills_per_week["user_int"] = 95 - bills_per_week["week_oldest_bill"]
bills_per_week

,user_id,week_last_bill,item_cnt,week_oldest_bill,user_int
0,1,95,1589,8,87
1,2,90,655,15,80
2,3,92,889,17,78
3,4,90,301,16,79
4,5,85,219,13,82
...,...,...,...,...,...
2494,2496,95,1420,17,78
2495,2497,95,1938,12,83
2496,2498,95,833,16,79
2497,2499,92,969,11,84


In [68]:
bills_per_week["items_per_week"] = bills_per_week["item_cnt"] / bills_per_week["user_int"]
bills_per_week.head()

,user_id,week_last_bill,item_cnt,week_oldest_bill,user_int,items_per_week
0,1,95,1589,8,87,18.264368
1,2,90,655,15,80,8.187500
2,3,92,889,17,78,11.397436
3,4,90,301,16,79,3.810127
4,5,85,219,13,82,2.670732


In [69]:
user_features = user_features.merge(bills_per_week[["user_id", "items_per_week"]], on='user_id', how='left')
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id,bill_avg,week_last_bill,items_per_week
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,2.492077,95,18.264368
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,2.673405,95,11.780220


## Сохраняем предобработанные данные

In [70]:
data.to_csv("../data/data.csv", index=False)
user_features.to_csv("../data/user_features.csv", index=False)
item_features.to_csv("../data/item_features.csv", index=False)